<a href="https://colab.research.google.com/github/Pelmenoff/data_science/blob/main/hw11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the necessary libraries
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, LSTM, Bidirectional, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Setting random seed for reproducibility
np.random.seed(5050)

# Loading the dataset
max_features = 20000  # Number of words to consider as features
maxlen = 100  # Cut texts after this number of words (among top max_features most common words)
batch_size = 32

# Load the data
print('Loading data...')
(x_data, y_data), (_, _) = imdb.load_data(num_words=max_features)
print(len(x_data), 'total sequences')

print('Pad sequences (samples x time)')
x_data = sequence.pad_sequences(x_data, maxlen=maxlen)
print('x_data shape:', x_data.shape)

# Split data into training, validation, and test sets
x_temp, x_test, y_temp, y_test = train_test_split(x_data, y_data, test_size=0.15, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.1765, random_state=42)  # 0.1765 * 0.85 ≈ 0.15

print('x_train shape:', x_train.shape)
print('x_val shape:', x_val.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 total sequences
Pad sequences (samples x time)
x_data shape: (25000, 100)
x_train shape: (17499, 100)
x_val shape: (3751, 100)
x_test shape: (3750, 100)


In [ ]:
# Function to compile and train model
def compile_and_train(model, epochs=10):
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_val, y_val))
    return history

# Simple RNN
print('Building Simple RNN model...')
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, 128))
model_rnn.add(SimpleRNN(128))
model_rnn.add(Dense(1, activation='sigmoid'))
print('Training Simple RNN model...')
compile_and_train(model_rnn)

# LSTM
print('Building LSTM model...')
model_lstm = Sequential()
model_lstm.add(Embedding(max_features, 128))
model_lstm.add(LSTM(128))
model_lstm.add(Dense(1, activation='sigmoid'))
print('Training LSTM model...')
compile_and_train(model_lstm)

# Bidirectional LSTM
print('Building Bidirectional LSTM model...')
model_bilstm = Sequential()
model_bilstm.add(Embedding(max_features, 128))
model_bilstm.add(Bidirectional(LSTM(128)))
model_bilstm.add(Dense(1, activation='sigmoid'))
print('Training Bidirectional LSTM model...')
compile_and_train(model_bilstm)

# Deep LSTM
print('Building Deep LSTM model...')
model_deep_lstm = Sequential()
model_deep_lstm.add(Embedding(max_features, 128))
model_deep_lstm.add(LSTM(128, return_sequences=True))
model_deep_lstm.add(LSTM(128))
model_deep_lstm.add(Dense(1, activation='sigmoid'))
print('Training Deep LSTM model...')
compile_and_train(model_deep_lstm)

Loading data...
25000 total sequences
Pad sequences (samples x time)
x_data shape: (25000, 100)
x_train shape: (17499, 100)
x_val shape: (3751, 100)
x_test shape: (3750, 100)
Building Simple RNN model...
Training Simple RNN model...
Epoch 1/10
547/547 [==============================] - 60s 107ms/step - loss: 0.6152 - accuracy: 0.6392 - val_loss: 0.5263 - val_accuracy: 0.7438
Epoch 2/10
547/547 [==============================] - 57s 105ms/step - loss: 0.4888 - accuracy: 0.7721 - val_loss: 0.5981 - val_accuracy: 0.6820
Epoch 3/10
547/547 [==============================] - 59s 108ms/step - loss: 0.4872 - accuracy: 0.7786 - val_loss: 0.5333 - val_accuracy: 0.7593
Epoch 4/10
547/547 [==============================] - 57s 103ms/step - loss: 0.3539 - accuracy: 0.8519 - val_loss: 0.5022 - val_accuracy: 0.7931
Epoch 5/10
547/547 [==============================] - 54s 99ms/step - loss: 0.3367 - accuracy: 0.8632 - val_loss: 0.5462 - val_accuracy: 0.7497
Epoch 6/10
547/547 [=======================

In [ ]:
test_loss1, test_acc1 = model_rnn.evaluate(x_test, y_test)
print(f'[Simple RNN] Test loss: {test_loss1}, accuracy: {test_acc1}')
test_loss2, test_acc2 = model_lstm.evaluate(x_test, y_test)
print(f'[LSTM] Test loss: {test_loss2}, accuracy: {test_acc2}')
test_loss3, test_acc3 = model_bilstm.evaluate(x_test, y_test)
print(f'[Bidirectional LSTM] Test loss: {test_loss3}, accuracy: {test_acc3}')
test_loss4, test_acc4 = model_deep_lstm.evaluate(x_test, y_test)
print(f'[Deep LSTM] Test loss: {test_loss4}, accuracy: {test_acc4}')

118/118 [==============================] - 2s 16ms/step - loss: 0.6416 - accuracy: 0.7523
[Simple RNN] Test loss: 0.6415784955024719, accuracy: 0.7522666454315186
118/118 [==============================] - 9s 76ms/step - loss: 1.0176 - accuracy: 0.8261
[LSTM] Test loss: 1.0176408290863037, accuracy: 0.8261333107948303
118/118 [==============================] - 14s 121ms/step - loss: 1.2418 - accuracy: 0.8296
[Bidirectional LSTM] Test loss: 1.241757869720459, accuracy: 0.8295999765396118
118/118 [==============================] - 16s 134ms/step - loss: 0.8028 - accuracy: 0.8187
[Deep LSTM] Test loss: 0.8027820587158203, accuracy: 0.8186666369438171


In [ ]:
model_bilstm_final = Sequential()
model_bilstm_final.add(Embedding(max_features, 128))
model_bilstm_final.add(Bidirectional(LSTM(128)))
model_bilstm_final.add(Dense(1, activation='sigmoid'))


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

In [ ]:
def compile_and_train_best_adam(model, epochs=30):
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_val, y_val),
                        callbacks=[early_stopping, reduce_lr])
    return history

In [ ]:
compile_and_train_best_adam(model_bilstm_final)

Epoch 1/30
547/547 [==============================] - 257s 461ms/step - loss: 0.4430 - accuracy: 0.7930 - val_loss: 0.3552 - val_accuracy: 0.8488 - lr: 0.0010
Epoch 2/30
547/547 [==============================] - 250s 457ms/step - loss: 0.2173 - accuracy: 0.9157 - val_loss: 0.3741 - val_accuracy: 0.8408 - lr: 0.0010
Epoch 3/30
547/547 [==============================] - 250s 458ms/step - loss: 0.1116 - accuracy: 0.9621 - val_loss: 0.4194 - val_accuracy: 0.8374 - lr: 0.0010
Epoch 4/30
547/547 [==============================] - 251s 458ms/step - loss: 0.0567 - accuracy: 0.9799 - val_loss: 0.5614 - val_accuracy: 0.8416 - lr: 0.0010
Epoch 5/30
547/547 [==============================] - 251s 458ms/step - loss: 0.0264 - accuracy: 0.9917 - val_loss: 0.6270 - val_accuracy: 0.8368 - lr: 0.0010
Epoch 6/30
547/547 [==============================] - 254s 464ms/step - loss: 0.0175 - accuracy: 0.9941 - val_loss: 0.7837 - val_accuracy: 0.8395 - lr: 0.0010
Epoch 7/30
547/547 [==========================

In [ ]:
test_loss_1, test_acc_1 = model_bilstm_final.evaluate(x_test, y_test)
print(f'Adam test loss: {test_loss_1}, accuracy: {test_acc_1}')

118/118 [==============================] - 14s 120ms/step - loss: 0.3674 - accuracy: 0.8360
Adam test loss: 0.36740344762802124, accuracy: 0.8360000252723694


In [ ]:
model_bilstm_final2 = Sequential()
model_bilstm_final2.add(Embedding(max_features, 128))
model_bilstm_final2.add(Bidirectional(LSTM(128)))
model_bilstm_final2.add(Dense(1, activation='sigmoid'))

In [ ]:
def compile_and_train_best_nadam(model, epochs=30):
    model.compile(loss='binary_crossentropy',
                  optimizer='Nadam',
                  metrics=['accuracy'])
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_val, y_val),
                        callbacks=[early_stopping, reduce_lr])
    return history

In [11]:
compile_and_train_best_nadam(model_bilstm_final2)

Epoch 1/30
547/547 [==============================] - 259s 464ms/step - loss: 0.4208 - accuracy: 0.8017 - val_loss: 0.3604 - val_accuracy: 0.8483 - lr: 0.0010
Epoch 2/30
547/547 [==============================] - 249s 454ms/step - loss: 0.2077 - accuracy: 0.9203 - val_loss: 0.3848 - val_accuracy: 0.8435 - lr: 0.0010
Epoch 3/30
547/547 [==============================] - 258s 472ms/step - loss: 0.1025 - accuracy: 0.9634 - val_loss: 0.4792 - val_accuracy: 0.8336 - lr: 0.0010
Epoch 4/30
547/547 [==============================] - 259s 474ms/step - loss: 0.0483 - accuracy: 0.9833 - val_loss: 0.7029 - val_accuracy: 0.8336 - lr: 0.0010
Epoch 5/30
547/547 [==============================] - 258s 471ms/step - loss: 0.0264 - accuracy: 0.9917 - val_loss: 0.8315 - val_accuracy: 0.8315 - lr: 0.0010
Epoch 6/30
547/547 [==============================] - 259s 473ms/step - loss: 0.0152 - accuracy: 0.9957 - val_loss: 0.8239 - val_accuracy: 0.8323 - lr: 0.0010
Epoch 7/30
547/547 [==========================

In [12]:
test_loss_2, test_acc_2 = model_bilstm_final2.evaluate(x_test, y_test)
print(f'Nadam test loss: {test_loss_2}, accuracy: {test_acc_2}')

118/118 [==============================] - 14s 121ms/step - loss: 0.3567 - accuracy: 0.8464
Nadam test loss: 0.35671666264533997, accuracy: 0.8464000225067139
